In [ ]:
import os
import numpy as np
from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt
import pypianoroll as ppr

# from midi_io import *

In [ ]:

programs = [0, 0, 24, 32, 48, 64, 80, 88]
is_drums = [True, False, False, False, False, False, False, False]
track_num = len(programs)
#track_names = ['Drums', 'Piano', 'Guitar', 'Bass', 'String']
track_names = ['Drums', 'Piano', 'Guitar', 'Bass', 'Ensemble',
               'Reed', 'Synth Lead','Synth Pad']
# read dataset
dataset_path = '/Users/mac/Desktop/Brain/MuseGAN/training_data/lastfm_alternative_8b_phrase.npy'

#dataset_path = '/Users/mac/Downloads/lpd/lpd_cleansed/B/B/C/TRBBCJH128F930C970/9b26da87aff1395e17e10610a6dbff78.npz'

dataset = np.load(dataset_path)
dataset_name = 'lastfm_alternative_8b_phrase'


In [ ]:
#track_names = ['bass', 'drums', 'guitar', 'strings', 'piano',]
track_num = 5
track_names = ['Drums', 'Piano', 'Guitar', 'Bass', 'String']
programs = [0, 0, 24, 32, 48]
is_drums = [True, False, False, False, False]
# track_names = ['Drums', 'Piano', 'Guitar','Bass', 'Ensemble', 'Reed', 'Synth Lead', 'Synth Pad']
# read dataset
dataset_path = '/Users/mac/Desktop/Brain/MuseGAN/training_data/lastfm_alternative_5b_phrase.npy'
dataset = np.load(dataset_path)
dataset_name = 'lastfm_alternative_5b_phrase'


In [ ]:
def ith_song_to_midi(dataset, dataset_name, track_num, 
                     programs, is_drums, track_names, index, midi_dir):
    now_song = dataset[index][0]
    now_song = np.reshape(now_song, (4*96, 84, track_num), 'C')
    
    tracks = []
    
    for i in range(track_num):
        tracks.append(now_song[:,:,i])
    
    padded_tracks = []

    for i in range(track_num):
        temp_pr = np.zeros((len(tracks[i]),128))
        # 肯定存在简单写法
        for x in range(len(tracks[i])):
            for y in range(len(tracks[i][0])):
                if tracks[0][x][y] == True:
                    temp_pr[x][y+24] = 100
        padded_tracks.append(temp_pr)

    padded_tracks = np.array(padded_tracks)
    
    print(padded_tracks.shape)
    
    music_tracks = []
    for i in range(track_num):
        temp_track = Track(    pianoroll=padded_tracks[i], 
                               program=programs[i], 
                               is_drum = is_drums[i], 
                               name=track_names[i])
        music_tracks.append(temp_track)
    
    music_tracks = np.array(music_tracks)
    multitrack = Multitrack(tracks=music_tracks, tempo=120.0,
                        # downbeat=[0, 96, 192, 288], 
                        beat_resolution=24)
    
    multitrack.write(os.path.join(midi_dir, dataset_name+'_'+str(index)+'.mid'))
    

In [ ]:
for i in range(10,11):
    ith_song_to_midi(dataset = dataset, 
                     dataset_name = dataset_name,
                     track_num = track_num, 
                     programs = programs, 
                     is_drums = is_drums,
                     track_names = track_names, 
                     index = i,
                     midi_dir = '/Users/mac/Desktop/test_ppr')

#### 

In [2]:
def write_midi(filepath, pianorolls, program_nums=None, is_drums=None,
               track_names=None, velocity=100, tempo=120.0, beat_resolution=24):
    """
    Write the given piano-roll(s) to a single MIDI file.

    Arguments
    ---------
    filepath : str
        Path to save the MIDI file.
    pianorolls : np.array, ndim=3
        The piano-roll array to be written to the MIDI file. Shape is
        (num_timestep, num_pitch, num_track).
    program_nums : int or list of int
        MIDI program number(s) to be assigned to the MIDI track(s). Available
        values are 0 to 127. Must have the same length as `pianorolls`.
    is_drums : list of bool
        Drum indicator(s) to be assigned to the MIDI track(s). True for
        drums. False for other instruments. Must have the same length as
        `pianorolls`.
    track_names : list of str
        Track name(s) to be assigned to the MIDI track(s).
    """
    if not np.issubdtype(pianorolls.dtype, np.bool_):
        raise TypeError("Support only binary-valued piano-rolls")
    if isinstance(program_nums, int):
        program_nums = [program_nums]
    if isinstance(is_drums, int):
        is_drums = [is_drums]

    if pianorolls.shape[2] != len(program_nums):
        raise ValueError("`pianorolls` and `program_nums` must have the same"
                         "length")
    if pianorolls.shape[2] != len(is_drums):
        raise ValueError("`pianorolls` and `is_drums` must have the same"
                         "length")
    if program_nums is None:
        program_nums = [0] * len(pianorolls)
    if is_drums is None:
        is_drums = [False] * len(pianorolls)

    multitrack = Multitrack(beat_resolution=beat_resolution, tempo=tempo)
    for idx in range(pianorolls.shape[2]):
        if track_names is None:
            track = Track(pianorolls[..., idx], program_nums[idx],
                          is_drums[idx])
        else:
            track = Track(pianorolls[..., idx], program_nums[idx],
                          is_drums[idx], track_names[idx])
        multitrack.append_track(track)
    multitrack.write(filepath)

In [3]:
def save_midi(filepath, phrases, config):
    """
    Save a batch of phrases to a single MIDI file.

    Arguments
    ---------
    filepath : str
        Path to save the image grid.
    phrases : list of np.array
        Phrase arrays to be saved. All arrays must have the same shape.
    pause : int
        Length of pauses (in timestep) to be inserted between phrases.
        Default to 0.
    """
    if not np.issubdtype(phrases.dtype, np.bool_):
        raise TypeError("Support only binary-valued piano-rolls")

    reshaped = phrases.reshape(-1, phrases.shape[1] * phrases.shape[2],
                               phrases.shape[3], phrases.shape[4])
    pad_width = ((0, 0), (0, config['pause_between_samples']),
                 (config['lowest_pitch'],
                  128 - config['lowest_pitch'] - config['num_pitch']),
                 (0, 0))
    padded = np.pad(reshaped, pad_width, 'constant')
    pianorolls = padded.reshape(-1, padded.shape[2], padded.shape[3])

    write_midi(filepath, pianorolls, config['programs'], config['is_drums'],
               tempo=config['tempo'])


In [4]:
programs = [0, 0, 24, 32, 48, 64, 80, 88]
is_drums = [True, False, False, False, False, False, False, False]
track_num = len(programs)
#track_names = ['Drums', 'Piano', 'Guitar', 'Bass', 'String']
track_names = ['Drums', 'Piano', 'Guitar', 'Bass', 'Ensemble',
               'Reed', 'Synth Lead','Synth Pad']
# read dataset
dataset_path = '/Users/mac/Desktop/Brain/MuseGAN/training_data/lastfm_alternative_8b_phrase.npy'

#dataset_path = '/Users/mac/Downloads/lpd/lpd_cleansed/B/B/C/TRBBCJH128F930C970/9b26da87aff1395e17e10610a6dbff78.npz'

dataset = np.load(dataset_path)
dataset_name = 'lastfm_alternative_8b_phrase'

In [7]:
song = dataset[1000]
config = {
    'programs': programs, 
    'is_drums': is_drums,
    'tempo':120,
    # Playback
    'pause_between_samples': 96,
    'tempo': 90.,
    # Data
    'num_bar': 4,
    'num_beat': 4,
    'num_pitch': 84,
    'num_track': 8,
    'num_timestep': 96,
    'beat_resolution': 24,
    'lowest_pitch': 24
    # MIDI note number of the lowest pitch in data tensors
}

save_midi('/Users/mac/Desktop/test_ppr/test_musegan_8b_1000.mid', song, config)

In [8]:
song.shape

(6, 4, 96, 84, 8)